Things to do:
1. [ ] The get_contibution methods currently use a mix of predict_proba and predict. This should be fixed - expect probs, and calculate from there.
2. [x] Compare the runtime of get_contributions and get_contributions_slow. Check on long signal windows. If the fast version isn't faster, use the slow version (its more user friendly). **the faster script is faster, and confirmed to work**
3. [ ] Try this tutorial script with a multivariate dataset
4. [ ] Try this tutorial with unequal lengths (like japanese vowels)

Multivariate options:
https://www.sktime.org/en/stable/api_reference/auto_generated/sktime.datasets.load_basic_motions.html#sktime.datasets.load_basic_motions
https://www.sktime.org/en/stable/api_reference/auto_generated/sktime.datasets.load_japanese_vowels.html#sktime.datasets.load_japanese_vowels



fast run - 796

In [1]:
from sktime.datasets import load_arrow_head, load_japanese_vowels  # univariate dataset, multivariate variable length
X_train_orig, y_train_orig = load_arrow_head(split="train", return_X_y=True)
X_test_orig, y_test_orig = load_arrow_head(split="test", return_X_y=True)

X_train_multi, y_train_multi = load_japanese_vowels(split="train", return_X_y=True)
X_test_multi, y_test_multi = load_japanese_vowels(split="test", return_X_y=True)

In [2]:
X_train_orig

,dim_0
0,0 -1.963009 1 -1.957825 2 -1.95614...
1,0 -1.774571 1 -1.774036 2 -1.77658...
2,0 -1.866021 1 -1.841991 2 -1.83502...
3,0 -2.073758 1 -2.073301 2 -2.04460...
4,0 -1.746255 1 -1.741263 2 -1.72274...
5,0 -1.982806 1 -1.978861 2 -1.93733...
6,0 -2.083023 1 -2.092058 2 -2.04946...
7,0 -1.633596 1 -1.643175 2 -1.61367...
8,0 -1.717013 1 -1.728059 2 -1.68330...
9,0 -2.245295 1 -2.223850 2 -2.17187...


In [3]:
from sktime.classification.shapelet_based import ShapeletTransformClassifier

model = ShapeletTransformClassifier()
model.fit(X_train_orig, y_train_orig)

model.score(X_test_orig, y_test_orig)

0.7885714285714286

In [ ]:
import importlib
import pyreal.explainers.time.occlusion_importance as oi
importlib.reload(oi)
import time

transformers = []

explainer = oi.OcclusionSaliency(model, X_train_orig, window_size=3, classes=['0', '1', '2'])
explainer.fit()
start = time.time()
explanation1 = explainer.produce(X_train_orig.iloc[0][0])
explanation2 = explainer.produce(X_train_orig.iloc[1][0])
print("total time: ", time.time() - start)

running fast
running fast


In [ ]:
import pyreal.utils.visualize as vis
importlib.reload(vis)

vis.plot_timeseries_saliency(X_train_orig.iloc[0][0], explanation1[0][0:245])
vis.plot_timeseries_saliency(X_train_orig.iloc[0][0], explanation2[0][0:245])
